In [51]:
import subprocess
package_cmd = "remove package firefox"
service_cmd = "start service firewalld"
user_cmd = "remove user harry"

#voicetext = package_cmd
#voicetext = service_cmd
voicetext = user_cmd


if "package" in voicetext:
    task = "package"
elif "service" in voicetext:
    task = "service"
elif "user" in voicetext:
    task = "user"

exec(task+"_manager(voicetext)")

In [22]:
def package_manager(voicetext):
    text = voicetext.strip().lower().split()
    name = text[text.index("package")+1]
    state = text[text.index("package")-1]
    if state in ("install", "present", "download"):
        state = "present"
    else:
        state = "absent"

    ansible_data = '''
    - hosts: localhost
      tasks:
        - name: "Handling package... "
          package:
            name: "{0}"
            state: "{1}"
    '''.format(name, state)

    with open("../playbooks/package_manager.yml","w") as f:
        f.write(ansible_data)

    output = subprocess.getoutput("ansible-playbook ../playbooks/package_manager.yml")
    return(output)

In [18]:
def service_manager(voicetext):
    text = voicetext.strip().lower().split()
    name = text[text.index("service")+1]
    state = text[text.index("service")-1]
    
    if state in ("start", "started"):
        state = "started"
    elif state in ("stopped","stop"):
        state = "stopped"
    elif state in ("restart", "restarted"):
        state = "restarted"
        
        
    ansible_data = '''
    - hosts: localhost
      tasks:
        - name: "Handling Service... "
          service:
            name: "{0}"
            state: "{1}"
    '''.format(name, state)

    with open("../playbooks/service_manager.yml","w") as f:
        f.write(ansible_data)

    output = subprocess.getoutput("ansible-playbook ../playbooks/service_manager.yml")
    return(output)

In [41]:
def user_manager(voicetext): #create user harry
    text = voicetext.strip().lower().split()
    name = text[text.index("user")+1]
    action = text[text.index("user")-1]
    
    
    if action in ("create", "add"):
        ansible_data = '''
- hosts: localhost
  tasks:
    - name: "Adding User..."
      user:
        name: "{}"
        generate_ssh_key: yes
        ssh_key_bits: 2048
        ssh_key_file: .ssh/id_rsa
        '''.format(name)
    elif action in ("remove", "delete"):
        ansible_data = '''
- hosts: localhost
  tasks:
    - name: "removing User..."
      user:
        name: "{}"
        state: absent
        remove: yes
        '''.format(name)
        
    with open("../playbooks/user_manager.yml","w") as f:
        f.write(ansible_data)

    output = subprocess.getoutput("ansible-playbook ../playbooks/user_manager.yml")
    return(output)

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'

PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [removing User...] ********************************************************
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

